This notebook aims to download the news relevant to Adobe from Jan 2020 to July 2025 and convert the news to quantitative features with LLM.

## 1. Import the libraries.

In [107]:
import pandas as pd
import glob
import json
from tqdm import tqdm

#News download
from gnews import GNews

# AI Models & LLM Integration
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

## 2. Download the news 
Download Adobe news (bi-monthly) using GNews and save the results as CSV files. The download process combines automated and manual steps to comply with the API's limit of 100 results per call while balancing efficiency with ongoing human verification.

In [19]:
for i in [2020, 2021, 2022, 2023, 2024]: #, 2025]:
    google_news = GNews(
        language='en',
        country='US',
        start_date = (i, 11, 2),
        end_date = (i, 12, 31),
        max_results=100,
    )

    Adobe_news = google_news.get_news('Adobe')

    df = pd.DataFrame({'Title': [item['title'] for item in Adobe_news],'Date': [item['published date'] for item in Adobe_news]})

    name = 'Adobe'+str(i)+'1112.csv'

    df.to_csv('./data/news/' + name)

    print(name)

Adobe20201112.csv
Adobe20211112.csv
Adobe20221112.csv
Adobe20231112.csv
Adobe20241112.csv


## 3. Process the news titles
Re-import the csv files, merge them into one dataframe, and resolve the issue of weekend news alignment.

### 3.1 Import the csv files and merge them

In [112]:
news_list = glob.glob("./data/news/Adobe*.csv")
news_dataframe = []

In [113]:
for file in news_list:
    df = pd.read_csv(file)  # Read each CSV
    df['Title'] = df['Title'].astype(str)  # Ensure 'Title' is a string
    df['Date'] = pd.to_datetime(df['Date'], format='%a, %d %b %Y %H:%M:%S %Z').dt.date  # Convert 'date' to datetime
    news_dataframe.append(df)  # Append to list

In [114]:
df_news = pd.DataFrame()
df_news = pd.concat(news_dataframe, ignore_index=True).drop(columns=['Unnamed: 0']).drop_duplicates().reset_index(drop=True)

In [115]:
df_news_per_day = df_news.groupby('Date')['Title'].apply(lambda x: ' | '.join(x)).reset_index()

Check the merged data.

In [116]:
df_news_per_day

,Date,Title
0,2020-01-01,Public Art Collection - City of San Jose (.gov...
1,2020-01-03,Adobe Animal Hospital Gets a New Owner - Today...
2,2020-01-04,Top Desktop Publishing Software for Windows - ...
3,2020-01-05,Empower Creativity with RTX Studio Products - ...
4,2020-01-06,How IMM is Empowering Students to Become the M...
...,...,...
1147,2025-06-26,Maximize creative efficiency with TikTok Symph...
1148,2025-06-27,Learn how to future-proof your creative career...
1149,2025-06-28,"SLO’s oldest home, La Loma Adobe, has a long, ..."
1150,2025-06-29,Adobe Posts Record Q2 Revenue and Raises Outlo...


All the news titles are grouped by dates from 1 Jan 2020 to 1 July 2025.

### 3.2 Weekend news alignment

Next, since some news was published over the weekend, in order not to miss them in the later data integration, I will merge these entries into the following Monday to better leverage their value.

In [117]:
#Create a dataframe that only store the weekend news.
df_news_per_day['Date'] = pd.to_datetime(df_news_per_day['Date'])
df_weekend = df_news_per_day[df_news_per_day['Date'].dt.weekday >= 5]

In [118]:
df_weekend 

,Date,Title
2,2020-01-04,Top Desktop Publishing Software for Windows - ...
3,2020-01-05,Empower Creativity with RTX Studio Products - ...
10,2020-01-18,Adobe repair with bluegrass accompaniment - Sa...
25,2020-02-08,A Better Way to Embed PDF Documents in Web Pag...
30,2020-02-15,"Photographers, This is Why Other Photographers..."
...,...,...
1126,2025-05-11,I ditched Canva and every other design app for...
1137,2025-06-14,Adobe: Selloff After Non-Event Earnings Means ...
1143,2025-06-21,"Adobe a ‘long-term AI winner’, says DA Davidso..."
1149,2025-06-28,"SLO’s oldest home, La Loma Adobe, has a long, ..."


Process the weekend alignment.

In [119]:
# Process each weekend row
for idx, row in df_weekend.iterrows():
    weekend_date = row['Date']
    title = row['Title']
    
    # Find the next Monday
    days_to_monday = 7 - weekend_date.weekday()
    next_monday = weekend_date + pd.Timedelta(days=days_to_monday)
    
    # Check if this Monday is already in news_df
    if next_monday in df_news_per_day['Date'].values:
        # Append the title to existing Monday with ' | '
        df_news_per_day.loc[df_news_per_day['Date'] == next_monday, 'Title'] = (
            df_news_per_day.loc[df_news_per_day['Date'] == next_monday, 'Title'] + ' | ' + title
        )
    else:
        # Insert new row with this Monday and the weekend title
        df_news_per_day = pd.concat([
            df_news_per_day,
            pd.DataFrame({'Date': [next_monday], 'Title': [title]})
        ], ignore_index=True)

# Remove the original weekend rows
df_news_per_day = df_news_per_day[df_news_per_day['Date'].dt.weekday < 5]

# Sort and reset index
df_news_per_day = df_news_per_day.sort_values('Date').reset_index(drop=True)

In [120]:
df_news_per_day

,Date,Title
0,2020-01-01,Public Art Collection - City of San Jose (.gov...
1,2020-01-03,Adobe Animal Hospital Gets a New Owner - Today...
2,2020-01-06,How IMM is Empowering Students to Become the M...
3,2020-01-09,4 Design Trends That Will Define 2020 - Adobe ...
4,2020-01-10,"Inside Adobe’s Colorful, Redesigned Headquarte..."
...,...,...
1063,2025-06-25,"How Adobe Express helps teams create smarter, ..."
1064,2025-06-26,Maximize creative efficiency with TikTok Symph...
1065,2025-06-27,Learn how to future-proof your creative career...
1066,2025-06-30,"SLO’s oldest home, La Loma Adobe, has a long, ..."


Check the adjusted dataframe by reviewing titles of a Monday and its previous weekends.

In [121]:
df_news_per_day['Title'][2]

"How IMM is Empowering Students to Become the Masters of Their Own Digital Learning - Adobe | Cloud Patent-Holding Firm Sued By Microsoft Is Expanding IP Arsenal - CRN Magazine | Acer Unveils 'Transforming' ConceptD 7 Ezel Laptop with Wacom Pen Support and 100% Adobe RGB - PetaPixel | Top Desktop Publishing Software for Windows - ThoughtCo | Empower Creativity with RTX Studio Products - NVIDIA Blog | Adobe Stock Sets New High on Earnings but Is Extremely Overbought - Investopedia"

In [122]:
df_weekend ['Title'][2]

'Top Desktop Publishing Software for Windows - ThoughtCo'

In [123]:
df_weekend ['Title'][3]

'Empower Creativity with RTX Studio Products - NVIDIA Blog | Adobe Stock Sets New High on Earnings but Is Extremely Overbought - Investopedia'

Weekend news issues have been addressed by merging them into the following Monday. Since public holidays vary from year to year and make up a relatively small portion of the calendar, news published on those days will remain untouched. These entries will be excluded later when the news dataset is merged with the main trading data.

## 4. Use AI for sentiment analysis
To extract the new titles' full value and transform them into meaningful features, I will use an AI LLM to interpret the content as numeric values. Specifically, for all news titles from a single media source on a given day, they will be converted into the following features:
- **Sentiment Score**: indicates whether the news titles expressed a positive and negative feeling that could impact Adobe's price. The value is between -1 and 1, and -1 means highly negative, 0 means nuetral while 1 means highly positive.

- **Emotional Intensity**: determines if the titles' emotion is high or Low. The value ranges from 0 to 1, and 0 means extremely low while 1 means extremely high.

The LLM was set up with my favorite GPT-4.1-Mini (2025-04-14) for its strong cost-efficiency.

Note: The following code may not execute, as the API key was removed after all the OpenAI calls. To proceed with re-running the code, **please follow the notebook instructions and import the checkpoint file at the next next cell**.

### 4.1 Create an LLM model and use it to perform sentiment analysis

Initiate the LLM model. API key is removed here.

In [124]:
openai_api_key = ""
llm = ChatOpenAI(model="gpt-4.1-mini-2025-04-14", openai_api_key=openai_api_key)

Define the function to analyze the news with the LLM model.

In [125]:
def analyze_news(news_titles):
    prompt = f"""
    You are a professional quant focusing on using news titles to predict next-day stock movement.

    **Company:** Adobe, an American software company that offers a wide range of programs from web design tools, photo manipulation and vector creation, through to video/audio editing, mobile app development, print layout and animation software.
    
    Your task is to analyze a single day's Adobe news titles, separated by '|', to extract **quantitative features** relevant for predicting Adobe's next-day price movement.
    
    Treat each title independently.
    
    For the given headlines, output the following scores:
    
    1. "sentiment score" — Reflects the emotional tone of the headlines and their potential directional impact on Adobe’s stock.  
       - Numeric value from -1 (very negative) to +1 (very positive); 0 means neutral.
    
    2. "emotion intensity" — Measures how emotionally charged the headlines are.  
       - Numeric value from 0 (very low intensity) to 1 (very high intensity).
    
    Return only a valid JSON object formatted like:
    
    {{
      "sentiment score": 0.32,
      "emotion intensity": 0.76
    }}
    """
    response = llm.invoke([HumanMessage(content=prompt.replace('\n', ''))])
    return response.content

Use the function to analyze the news titles and output results in the 2 newly generated columns.

In [137]:
# Initialize columns
df_news_per_day['sentiment score'] = None
df_news_per_day['emotion intensity'] = None

# Apply analysis row-by-row
for idx in tqdm(df_news_per_day.index):
    titles = df_news_per_day.at[idx, 'Title']
    try:
        result_json = analyze_news(titles)
        result = json.loads(result_json)
        df_news_per_day.at[idx, 'sentiment score'] = result.get('sentiment score')
        df_news_per_day.at[idx, 'emotion intensity'] = result.get('emotion intensity')
    except Exception as e:
        print(f"Error processing index {idx}: {e}")

07/20/2025 10:57:05 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:57:06 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:57:08 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:57:09 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:57:10 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:57:11 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:57:12 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
  1%|▎                                         | 7/1068 [00:09<20:00,  1.13s/it]

Error processing index 6: Expecting value: line 1 column 1 (char 0)


07/20/2025 10:57:13 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:57:14 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:57:15 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:57:17 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:57:19 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:57:20 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:57:21 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:57:22 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:57:23 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:57:23 PM - HTTP Request: POST https://api

Error processing index 48: Expecting value: line 1 column 1 (char 0)


07/20/2025 10:58:00 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:58:01 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:58:02 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:58:03 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:58:04 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:58:05 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:58:06 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:58:08 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:58:09 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 10:58:10 PM - HTTP Request: POST https://api

...

07/20/2025 11:12:39 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:12:40 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:12:41 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:12:42 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:12:43 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:12:44 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:12:45 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:12:46 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:12:47 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:12:48 PM - HTTP Request: POST https://api

Error processing index 901: Expecting value: line 1 column 1 (char 0)


07/20/2025 11:13:24 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:13:25 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:13:26 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:13:28 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:13:29 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:13:30 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:13:31 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:13:32 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:13:33 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:13:34 PM - HTTP Request: POST https://api

Error processing index 912: Expecting value: line 1 column 1 (char 0)


07/20/2025 11:13:36 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:13:38 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:13:39 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:13:41 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:13:42 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:13:43 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:13:44 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:13:45 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:13:46 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:13:47 PM - HTTP Request: POST https://api

In [138]:
df_news_per_day

,Date,Title,sentiment score,emotion intensity
0,2020-01-01,Public Art Collection - City of San Jose (.gov...,0,0
1,2020-01-03,Adobe Animal Hospital Gets a New Owner - Today...,0.45,0.58
2,2020-01-06,How IMM is Empowering Students to Become the M...,0,0
3,2020-01-09,4 Design Trends That Will Define 2020 - Adobe ...,0.15,0.4
4,2020-01-10,"Inside Adobe’s Colorful, Redesigned Headquarte...",0,0
...,...,...,...,...
1063,2025-06-25,"How Adobe Express helps teams create smarter, ...",0.15,0.42
1064,2025-06-26,Maximize creative efficiency with TikTok Symph...,0.45,0.62
1065,2025-06-27,Learn how to future-proof your creative career...,0.0,0.0
1066,2025-06-30,"SLO’s oldest home, La Loma Adobe, has a long, ...",0.12,0.38


### 4.2 Review the output data from LLM and fix issues

Check if there is any missing values in the newly generated rows.

In [145]:
missing_rows_sentiment_score = df_news_per_day[df_news_per_day['sentiment score'].isnull()]
missing_rows_emotion_intensity = df_news_per_day[df_news_per_day['emotion intensity'].isnull()]

print(len(missing_rows_sentiment_score), len(missing_rows_emotion_intensity))

23 23


In [142]:
missing_rows_sentiment_score

,Date,Title,sentiment score,emotion intensity
6,2020-01-14,Adobe Makes Leadership Change To Grow Its Digi...,None,None
48,2020-03-18,Adobe gives free Creative Cloud access to stud...,None,None
112,2020-07-13,Adobe Subsidiary Expands Surface Design for 3D...,None,None
139,2020-08-26,"Salesforce Beats SAP, Oracle, Microsoft, Adobe...",None,None
145,2020-09-03,Adobe's First Creative Cloud Spot Invites You ...,None,None
147,2020-09-08,Finding the message in the edit with star edit...,None,None
149,2020-09-10,"Meet Marc Levoy, Adobe’s new VP and fellow - A...",None,None
165,2020-10-09,Why is Adobe Called Adobe? The Story of Garage...,None,None
256,2021-03-10,Pack more megapixels into your photos with Ado...,None,None
278,2021-04-13,"C++ at Adobe with Sean Parent, senior principa...",None,None


There are 23 rows missing both sentiment score and emotion intensity.

Create a function to execute the analyze_news functino safely, and use it to address the missing value problem.

In [148]:
def safe_analyze_news(news_titles, max_retries=3):
    for attempt in range(max_retries):
        try:
            result_json = analyze_news(news_titles)
            return json.loads(result_json)
        except Exception as e:
            print(f"Retry {attempt + 1} failed: {e}")
            time.sleep(2)
    return {"sentiment score": None, "emotion intensity": None}

In [149]:
for idx in missing_rows_sentiment_score.index:
    titles = df_news_per_day.at[idx, 'Title']
    result = safe_analyze_news(titles)
    df_news_per_day.at[idx, 'sentiment score'] = result.get('sentiment score')
    df_news_per_day.at[idx, 'emotion intensity'] = result.get('emotion intensity')

07/20/2025 11:27:54 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:27:55 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:27:56 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:27:57 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:27:57 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:27:59 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:28:00 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:28:01 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:28:03 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
07/20/2025 11:28:03 PM - HTTP Request: POST https://api

The fix is done. Check the rows where sentiment score and emotion intensity were missing before.

In [155]:
df_news_per_day.loc[missing_rows_sentiment_score.index]

,Date,Title,sentiment score,emotion intensity
6,2020-01-14,Adobe Makes Leadership Change To Grow Its Digi...,0.45,0.58
48,2020-03-18,Adobe gives free Creative Cloud access to stud...,0.45,0.58
112,2020-07-13,Adobe Subsidiary Expands Surface Design for 3D...,0.45,0.58
139,2020-08-26,"Salesforce Beats SAP, Oracle, Microsoft, Adobe...",0,0
145,2020-09-03,Adobe's First Creative Cloud Spot Invites You ...,0.45,0.58
147,2020-09-08,Finding the message in the edit with star edit...,0.15,0.4
149,2020-09-10,"Meet Marc Levoy, Adobe’s new VP and fellow - A...",0.45,0.58
165,2020-10-09,Why is Adobe Called Adobe? The Story of Garage...,0,0
256,2021-03-10,Pack more megapixels into your photos with Ado...,0.45,0.6
278,2021-04-13,"C++ at Adobe with Sean Parent, senior principa...",0,0


In [184]:
missing_rows_sentiment_score = df_news_per_day[df_news_per_day['sentiment score'].isnull()]
missing_rows_emotion_intensity = df_news_per_day[df_news_per_day['emotion intensity'].isnull()]

print(len(missing_rows_sentiment_score), len(missing_rows_emotion_intensity))

0 0


## 5. Export the result
Save the dataframe in csv format, and we will continue to use it in the Final Project's main notebook.

In [183]:
df_news_per_day.to_csv("./data/news_with_sentiment_filled.csv", index=False)